In [11]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [12]:
# awk -F "\t" 'NF != 11' ronec_no_metadata.conllup
# sed '/^#/ d' ronec.conllup > ronec_no_metadata.conllup

In [13]:
ronec = pd.read_csv('/Users/apum/ronec/ronec/conllup/raw/ronec_no_metadata.conllup', sep='\t', header=None)

In [14]:
ronec.head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,1,Tot,tot,ADV,Rp,_,3,advmod,_,_,*
1,2,în,în,ADP,Spsa,AdpType=Prep|Case=Acc,3,case,_,_,*
2,3,cadrul,cadru,NOUN,Ncmsry,"Case=Acc,Nom|Definite=Def|Gender=Masc|Number=Sing",10,obl,_,_,*
3,4,etapei,etapă,NOUN,Ncfsoy,"Case=Dat,Gen|Definite=Def|Gender=Fem|Number=Sing",3,nmod,_,_,*
4,5,a,al,DET,Tsfs,Gender=Fem|Number=Sing|Poss=Yes|PronType=Prs,6,det,_,_,1:ORDINAL


In [15]:
# FORM: Word form or punctuation symbol.
# LEMMA: Lemma or stem of word form.
# UPOSTAG: Universal part-of-speech tag drawn from our revised version of the Google universal POS tags.
# XPOSTAG: Language-specific part-of-speech tag; underscore if not available.
ronec = ronec[[1, 2, 3, 4, 10]]

In [16]:
ronec = ronec.replace({10:'*'}, 'O')

In [17]:
ronec['NER'] = (np.where(ronec[10].str.contains(':'),
                  ronec[10].str.split(':').str[1],
                  ronec[10]))

ronec['NER_aux'] = (np.where(ronec[10].str.contains(':'),
                  ronec[10].str.split(':').str[0],
                  ronec[10]))
ronec.fillna('O', inplace=True)

In [18]:
ronec.head()

,1,2,3,4,10,NER,NER_aux
0,Tot,tot,ADV,Rp,O,O,O
1,în,în,ADP,Spsa,O,O,O
2,cadrul,cadru,NOUN,Ncmsry,O,O,O
3,etapei,etapă,NOUN,Ncfsoy,O,O,O
4,a,al,DET,Tsfs,1:ORDINAL,ORDINAL,1


In [19]:
ronec['NER_NER'] = np.where((ronec['NER'].str.len() > 1), ronec['NER'], '0')
ronec['NER_NER'] = np.where((ronec['NER'] == ronec['NER_aux'].shift(1)), ronec['NER'].shift(1), ronec['NER'])

In [20]:
ronec = ronec[[1, 2, 3, 4, 'NER_NER']]

In [21]:
ronec.head()

,1,2,3,4,NER_NER
0,Tot,tot,ADV,Rp,O
1,în,în,ADP,Spsa,O
2,cadrul,cadru,NOUN,Ncmsry,O
3,etapei,etapă,NOUN,Ncfsoy,O
4,a,al,DET,Tsfs,ORDINAL


In [22]:
ronec_train = pd.read_csv('/Users/apum/ronec/ronec/conllup/raw/train_ronec_no_metadata.conllubio', sep='\t', header=None)
ronec_dev = pd.read_csv('/Users/apum/ronec/ronec/conllup/raw/dev_ronec_no_metadata.conllubio', sep='\t', header=None)

In [23]:
ronec_train = ronec_train[[1, 2, 3, 7, 9]]
ronec_train.head()

,1,2,3,7,9
0,Tot,tot,ADV,advmod,O
1,în,în,ADP,case,O
2,cadrul,cadru,NOUN,obl,O
3,etapei,etapă,NOUN,nmod,O
4,a,al,DET,det,I ORDINAL


In [24]:
ronec_dev = ronec_dev[[1, 2, 3, 7, 9]]
ronec_dev.head()

,1,2,3,7,9
0,Totodată,totodată,ADV,advmod,O
1,",",",",PUNCT,punct,O
2,pierderea,pierdere,NOUN,nsubj,O
3,documentului,document,NOUN,nmod,O
4,de,de,ADP,case,O


In [25]:
ronec_train[9] = ronec_train[9].str.replace(' ', '-')
ronec_dev[9] = ronec_dev[9].str.replace(' ', '-')

In [26]:
ronec_train = ronec_train.dropna(axis=0, subset=[9])
ronec_dev = ronec_dev.dropna(axis=0, subset=[9])

In [27]:
#ronec_train.to_csv('/Users/apum/romanian-bert/downstream-tasks/ner/train_ronec.conllu', header=None, sep='\t', index=False)
#ronec_dev.to_csv('/Users/apum/romanian-bert/downstream-tasks/ner/dev_ronec.conllu', header=None, sep='\t', index=False)

In [28]:
ronec_train.isnull().sum()

1    0
2    0
3    0
7    0
9    0
dtype: int64